<a href="https://colab.research.google.com/github/talhaanwarch/OffenseEval2020/blob/master/Turkish/biGRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Turkish Language

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
cd /content/drive/My Drive/dataset/OffenseEval2020/data/Turkish/

/content/drive/My Drive/dataset/OffenseEval2020/data/Turkish


In [3]:
ls

offenseval-annotation.txt      readme-trainingset-tr.txt
offenseval-tr-training-v1.tsv


In [0]:
import pandas as pd
train=pd.read_csv( 'offenseval-tr-training-v1.tsv',sep="\t")
train=train.sample(frac=1)

In [5]:
train.head()

,id,tweet,subtask_a
20909,45871,@USER soda da tavsiye ederim suyun yerine,NOT
21853,11133,Ne yazarsam yazayim ustune kac kisinin kalbine...,NOT
24645,36124,7 sülaleni s..sin Galatasaray oruspu çocuğu bi...,OFF
21904,29601,"Bukowski'nin dediği gibi: ""Hayatta kimseyi ...",NOT
23858,37727,@USER Oh be sonunda bana dusulmesi için bir sebep,NOT


In [6]:
train.tail()

,id,tweet,subtask_a
26842,27505,"Gece acıkınca akşamdan kalmış sarmaları yemek,...",NOT
23071,14737,@USER Bu cocuk icmiyor ise gitmiyor hayati yok...,NOT
14225,25431,@USER Türkiye'nin en iyi hukuk fakültesini kaz...,OFF
4206,26844,böyle sana sarılıp hiç konuşmadan göğüs kafesi...,NOT
24424,42982,saat 11.19 ve ben keçiboynuzu pekmezi yiyorum...,NOT


In [0]:
labels=train['subtask_a']
train=train['tweet']
labels=pd.factorize(labels)[0]

In [8]:
import collections
collections.Counter(labels)

Counter({0: 25231, 1: 6046})

In [9]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
max_words = 10000 #frequency of words to be kept
max_len = 200

tokenize = Tokenizer(num_words=max_words)
tokenize.fit_on_texts(train)
sequences = tokenize.texts_to_sequences(train)
word_index = tokenize.word_index
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

Using TensorFlow backend.


In [10]:
len(sequences_matrix),len(labels)

(31277, 31277)

In [0]:
from keras.models import Sequential
from keras.layers import Embedding,CuDNNGRU,GRU,Dense,Dropout,Bidirectional,GlobalMaxPool1D,GlobalAveragePooling1D, SpatialDropout1D
from keras.optimizers import RMSprop
from keras.initializers import Constant
from sklearn.utils import class_weight
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [0]:
from keras import backend as K
def f1(y_true, y_pred):
    '''
    metric from here 
    https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
    '''
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
model = Sequential()
model.add(Embedding(input_dim=max_words,output_dim=300,input_length=max_len))
model.add(Bidirectional(CuDNNGRU(64, return_sequences = True)))
#model.add(Bidirectional(CuDNNGRU(30, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(50, activation="relu"))
model.add(Dense(20, activation="relu"))

model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=[f1])

In [0]:
import numpy as np
class_weights = class_weight.compute_class_weight('balanced',np.unique(labels),labels)
class_weights=dict(enumerate(class_weights))

In [0]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(sequences_matrix,labels,test_size=0.2,stratify=labels,shuffle=True)

In [0]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.000001, verbose=1)

In [78]:

model.fit(x_train,y_train,batch_size=256,epochs=10,verbose=2,class_weight=class_weights,validation_data=(x_val,y_val),callbacks=[reduce_lr])

Train on 25021 samples, validate on 6256 samples
Epoch 1/10
 - 5s - loss: 0.5856 - f1: 0.4672 - val_loss: 0.4715 - val_f1: 0.5583
Epoch 2/10
 - 5s - loss: 0.4033 - f1: 0.6476 - val_loss: 0.4890 - val_f1: 0.5383
Epoch 3/10
 - 5s - loss: 0.2569 - f1: 0.7678 - val_loss: 0.5551 - val_f1: 0.5138

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0004000000189989805.
Epoch 4/10
 - 5s - loss: 0.1121 - f1: 0.9010 - val_loss: 0.6598 - val_f1: 0.5233
Epoch 5/10
 - 5s - loss: 0.0701 - f1: 0.9389 - val_loss: 0.7358 - val_f1: 0.5201

Epoch 00005: ReduceLROnPlateau reducing learning rate to 8.000000379979611e-05.
Epoch 6/10
 - 5s - loss: 0.0461 - f1: 0.9620 - val_loss: 0.7594 - val_f1: 0.5166
Epoch 7/10
 - 5s - loss: 0.0418 - f1: 0.9648 - val_loss: 0.7860 - val_f1: 0.5150

Epoch 00007: ReduceLROnPlateau reducing learning rate to 1.6000001050997525e-05.
Epoch 8/10
 - 5s - loss: 0.0383 - f1: 0.9678 - val_loss: 0.7909 - val_f1: 0.5128
Epoch 9/10
 - 5s - loss: 0.0375 - f1: 0.9697 - val_loss: 0.

In [79]:
from sklearn.metrics import classification_report

y_pred = model.predict(x_val, batch_size=128, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
y_pred = (y_pred > 0.5)

print(classification_report(y_val, y_pred))

6256/6256 [==============================] - 1s 106us/step
              precision    recall  f1-score   support

           0       0.88      0.90      0.89      5047
           1       0.53      0.50      0.52      1209

    accuracy                           0.82      6256
   macro avg       0.71      0.70      0.70      6256
weighted avg       0.81      0.82      0.82      6256



In [0]:
import keras
keras.backend.clear_session()